In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torchtext import data 
import torchtext
from pathlib import Path
import pandas as pd
import spacy

In [2]:
device = "cuda"

In [4]:
!mkdir -p data
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/sentiment-analysis-is-bad/data/training.1600000.processed.noemoticon.csv.zip -P data
!unzip -n -d data data/training.1600000.processed.noemoticon.csv.zip

--2021-01-10 04:23:34--  https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/sentiment-analysis-is-bad/data/training.1600000.processed.noemoticon.csv.zip
Resolving nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)... 162.243.189.2
Connecting to nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)|162.243.189.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85088192 (81M) [application/zip]
Saving to: ‘data/training.1600000.processed.noemoticon.csv.zip’

training.1600000.pr 100%[===================>]  81.15M  37.8MB/s    in 2.1s    

2021-01-10 04:23:36 (37.8 MB/s) - ‘data/training.1600000.processed.noemoticon.csv.zip’ saved [85088192/85088192]

Archive:  data/training.1600000.processed.noemoticon.csv.zip
  inflating: data/training.1600000.processed.noemoticon.csv  


In [5]:
tweetsDF = pd.read_csv("./data/training.1600000.processed.noemoticon.csv", 
engine="python", header=None)

In [6]:
tweetsDF[0].value_counts()

4    800000
0    800000
Name: 0, dtype: int64

In [7]:
tweetsDF.head(5)

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [8]:
tweetsDF["sentiment_cat"] = tweetsDF[0].astype('category')
tweetsDF["sentiment"] = tweetsDF["sentiment_cat"].cat.codes
tweetsDF.to_csv("train-processed.csv", header=None, index=None)      
tweetsDF.sample(10000).to_csv("train-processed-sample.csv", header=None, index=None)

In [9]:
LABEL = data.LabelField()
TWEET = data.Field(tokenize='spacy', lower=True)

fields = [('score',None), ('id',None),('date',None),('query',None),
      ('name',None),
      ('tweet', TWEET),('category',None),('label',LABEL)]

In [10]:
#Create our Dataset and DataLoaders
twitterDataset = torchtext.data.TabularDataset(
        path="train-processed-sample.csv", 
        format="CSV", 
        fields=fields,
        skip_header=False)

In [11]:
(train, test, valid)=twitterDataset.split(split_ratio=[0.6,0.2,0.2],stratified=True, strata_field='label')

(len(train),len(test),len(valid))

(6000, 2000, 2000)

In [12]:
vocab_size = 20000
TWEET.build_vocab(train, max_size = vocab_size)
LABEL.build_vocab(train)
TWEET.vocab.freqs.most_common(10)

[('i', 3833),
 ('!', 3439),
 ('.', 2899),
 ('to', 2163),
 (' ', 2083),
 ('the', 1906),
 (',', 1900),
 ('a', 1400),
 ('my', 1177),
 ('and', 1176)]

In [13]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
(train, valid, test), 
batch_size = 32,
device = device,
sort_key = lambda x: len(x.tweet),
sort_within_batch = False)

In [21]:
#Our First LSTM
class OurFirstLSTM(nn.Module):
    def __init__(self, hidden_size, embedding_dim, vocab_size):
        super(OurFirstLSTM, self).__init__()
    
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.encoder = nn.LSTM(input_size=embedding_dim,  
                hidden_size=hidden_size, num_layers=1)
        self.predictor = nn.Linear(hidden_size, 2)

    def forward(self, seq):
        output, (hidden,_) = self.encoder(self.embedding(seq))
        preds = self.predictor(hidden.squeeze(0))
        return preds

model = OurFirstLSTM(100,300, 20002)
model.to(device)


OurFirstLSTM(
  (embedding): Embedding(20002, 300)
  (encoder): LSTM(300, 100)
  (predictor): Linear(in_features=100, out_features=2, bias=True)
)

In [22]:
#Training
optimizer = optim.Adam(model.parameters(), lr=2e-2)
criterion = nn.CrossEntropyLoss()

def train(epochs, model, optimizer, criterion, train_iterator, valid_iterator):
    for epoch in range(1, epochs + 1):
     
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch_idx, batch in enumerate(train_iterator):
            optimizer.zero_grad()
            predict = model(batch.tweet)
            loss = criterion(predict,batch.label)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * batch.tweet.size(0)
        training_loss /= len(train_iterator)
 
        
        model.eval()
        for batch_idx,batch in enumerate(valid_iterator):
            predict = model(batch.tweet)
            loss = criterion(predict,batch.label)
            valid_loss += loss.data.item() * batch.tweet.size(0)
 
        valid_loss /= len(valid_iterator)
        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}'.format(epoch, training_loss, valid_loss))

In [23]:
train(5, model, optimizer, criterion, train_iterator, valid_iterator)

Epoch: 1, Training Loss: 24.58, Validation Loss: 13.20
Epoch: 2, Training Loss: 23.72, Validation Loss: 15.32
Epoch: 3, Training Loss: 23.44, Validation Loss: 15.11
Epoch: 4, Training Loss: 22.90, Validation Loss: 15.80
Epoch: 5, Training Loss: 21.71, Validation Loss: 16.25


In [24]:
#Making Predictions
def classify_tweet(tweet):
  categories = {0:"Negative", 1:"Positive"}
  processed = TWEET.process([TWEET.preprocess(tweet)])
  processed = processed.to(device)
  model.eval()
  return categories[model(processed).argmax().item()]

In [25]:
#Data Augmentation
def random_deletion(words, p=0.5):
  if len(words)==1:
    return words
  remaining = list(filter(lambda x:random.uniform(0,1) > p,words))
  if len(remaining) == 0:
    return [random.choice(words)]
  else:
    return remaining

In [26]:
def random_swap(sentence, n=5):
    length = range(len(sentence))
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1]
    return sentence

In [27]:
# Note: you'll have to define remove_stopwords() and get_synonyms() elsewhere

def random_insertion(sentence,n):
    words = remove_stopwords(sentence)
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        sentence.insert(randrange(len(sentence)+1), new_synonym)
    return sentence

In [41]:
from google_trans_new import google_translator 
import random

translator = google_translator()

sentences = ['The cat sat on the mat']

translations_fr = translator.translate(sentences, lang_tgt='fr')
translations_en = translator.translate(translations_fr, lang_tgt='en')
print(translations_en)   

available_langs = list(googletrans.LANGUAGES.keys())
tr_lang = random.choice(available_langs)
print(f"Translating to {googletrans.LANGUAGES[tr_lang]}")

translations = translator.translate(sentences, lang_tgt=tr_lang)
print(translations)

translations_en_random = translator.translate(t_text, lang_tgt="en", lang_src=tr_lang)
print(translations_en_random)

['The cat sat on the carpet'] 
Translating to italian
['Il gatto si è seduto sul tappetino'] 
['[', "'",' K ',' u ',' c ',' i ',' n ',' g ',' ',' i ',' t ',' u ',' ' , 'd', 'u', 'd', 'u', 'k', '', 'd', 'i', '', 'a', 't', 'a', 's' , '', 't', 'i', 'k', 'a', 'r', "'",'] ',' '] 
